In [ ]:
import os
from anomalib.data import get_transforms
from anomalib.models import get_model
from anomalib.utils.callbacks import get_callbacks
from anomalib.utils.inference import Inferencer
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# 데이터 경로 설정
train_data_path = "path/to/train_dataset"  # 정상 이미지 폴더
test_data_path = "path/to/test_dataset"    # 테스트 이미지 폴더

# 데이터 전처리
transforms = get_transforms(image_size=256)

train_dataset = ImageFolder(root=train_data_path, transform=transforms["train"])
test_dataset = ImageFolder(root=test_data_path, transform=transforms["test"])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# GANomaly 모델 설정
model_config = {
    "name": "ganomaly",
    "image_size": 256,
    "latent_dim": 100,
    "batch_size": 32,
    "lr": 0.0002,
    "num_epochs": 50
}
model = get_model(model_config)

# 학습 콜백
callbacks = get_callbacks(
    model_name="ganomaly",
    log_dir="./logs"
)
callbacks.append(EarlyStopping(monitor="val_loss", patience=5, mode="min"))

# 모델 학습
trainer = Trainer(
    max_epochs=model_config["num_epochs"],
    gpus=1 if torch.cuda.is_available() else 0,
    callbacks=callbacks
)

trainer.fit(model, train_loader)

# 모델 저장
os.makedirs("saved_model", exist_ok=True)
trainer.save_checkpoint("saved_model/ganomaly.ckpt")

# 모델 로드 및 추론
inferencer = Inferencer(model="ganomaly", ckpt_path="saved_model/ganomaly.ckpt")

# 테스트 이미지 예측
for batch in test_loader:
    images, _ = batch
    predictions = inferencer.predict(images)
    for i, pred in enumerate(predictions):
        print(f"Image {i}: Anomaly Score = {pred['anomaly_score']}, Is Normal: {pred['is_normal']}")
